In [1]:
'''from google.colab import drive
drive.mount('/content/drive')
!pip install deepchem rdkit matplotlib networkx'''

import deepchem as dc
from rdkit import Chem
from rdkit.Chem import Draw
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from deepchem.feat import MolGraphConvFeaturizer
from deepchem.feat import CircularFingerprint
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from deepchem.feat import ConvMolFeaturizer
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
import deepchem as dc
from methods_fingerprints import *
#from drive.MyDrive.code.methods import *
#from drive.MyDrive.code.methods_fingerprints import *

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 35.9 MB/s eta 0:00:00


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


In [2]:
smiles_tr,smiles_test, logSweeter_tr, logSweeter_test, families_tr, families_test = load_data_remove("../data/SweetnersDB.xlsx")#"/content/drive/MyDrive/code/SweetnersDB.xlsx")
X_tr,X_test,scaler_X = prepare_fingerprint(smiles_tr,smiles_test)
y_tr,y_test,scaler_y = prepare_targets(logSweeter_tr,logSweeter_test)

Index(['Name', 'logSw', 'Chemical family', 'Dataset', 'Smiles'], dtype='object')


[15:18:44] DEPRECATION WARNING: please use MorganGenerator
[15:18:44] DEPRECATION WARNING: please use MorganGenerator
[15:18:44] DEPRECATION WARNING: please use MorganGenerator
[15:18:44] DEPRECATION WARNING: please use MorganGenerator
[15:18:44] DEPRECATION WARNING: please use MorganGenerator
[15:18:44] DEPRECATION WARNING: please use MorganGenerator
[15:18:44] DEPRECATION WARNING: please use MorganGenerator
[15:18:44] DEPRECATION WARNING: please use MorganGenerator
[15:18:44] DEPRECATION WARNING: please use MorganGenerator
[15:18:44] DEPRECATION WARNING: please use MorganGenerator
[15:18:44] DEPRECATION WARNING: please use MorganGenerator
[15:18:44] DEPRECATION WARNING: please use MorganGenerator
[15:18:44] DEPRECATION WARNING: please use MorganGenerator
[15:18:44] DEPRECATION WARNING: please use MorganGenerator
[15:18:44] DEPRECATION WARNING: please use MorganGenerator
[15:18:44] DEPRECATION WARNING: please use MorganGenerator
[15:18:44] DEPRECATION WARNING: please use MorganGenerat

In [3]:
onehot_tr,onehot_test = prepare_famille_chimique(families_tr, families_test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate

input_fingerprint = Input(shape=(X_tr.shape[1],), name='fingerprint_input')
input_famille = Input(shape=(onehot_tr.shape[1],), name='famille_input')

# Combiner les entrées
concat = Concatenate()([input_fingerprint, input_famille])

# Ajouter des couches denses
dense = Dense(512, activation='relu')(concat)
output = Dense(256, activation='relu')(dense)

# Créer le modèle
model = Model(inputs=[input_fingerprint, input_famille], outputs=output)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ fingerprint_input         │ (None, 1024)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ famille_input             │ (None, 9)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 1033)           │              0 │ fingerprint_input[0][… │
│                           │                        │                │ famille_input[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 512)            │        529,408 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 256)            │        131,328 │ dense[0][0]            │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 660,736 (2.52 MB)

 Trainable params: 660,736 (2.52 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# Faire une prédiction avec le modèle
predictions_tr = model.predict([X_tr, onehot_tr])
print("\nPrédictions :")
print(predictions_tr)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

Prédictions :
[[0.         0.         0.30067918 ... 0.02240904 0.         0.        ]
 [0.12108305 0.2037729  0.24556732 ... 0.         0.02892289 0.        ]
 [0.         0.24263072 0.10437249 ... 0.         0.         0.        ]
 ...
 [0.         0.02549958 0.95302284 ... 0.         0.13513121 0.        ]
 [1.7006503  0.         1.1265434  ... 0.11152619 0.1139127  0.2177144 ]
 [0.         0.8939491  0.5714333  ... 0.         0.         0.60108685]]


In [6]:
# Faire une prédiction avec le modèle
predictions_test = model.predict([X_test, onehot_test])
print("\nPrédictions :")
print(predictions_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 

Prédictions :
[[0.         0.3171934  0.06976386 ... 0.         0.         0.        ]
 [0.28584772 0.02486742 0.18908815 ... 0.         0.53696495 0.        ]
 [0.9059696  1.3384314  0.9408309  ... 0.         0.         0.33034122]
 ...
 [0.         0.8454875  0.         ... 0.         0.         0.        ]
 [0.46144247 1.2676347  2.8754714  ... 0.         0.         0.39422548]
 [2.359302   1.619616   2.0652442  ... 0.5897813  0.2832209  0.        ]]


# scoring log reg

In [8]:
accuracy_log,conf_matrix_log,class_report_log,y_pred_log,target_test_log = scoring(y_tr,y_test,predictions_tr,predictions_test,0,'LogReg')

Accuracy: 0.8064516129032258
Confusion Matrix:
[[14  7]
 [ 5 36]]
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.67      0.70        21
           1       0.84      0.88      0.86        41

    accuracy                           0.81        62
   macro avg       0.79      0.77      0.78        62
weighted avg       0.80      0.81      0.80        62



# scoring decision tree

In [11]:
accuracy_tree,conf_matrix_tree,class_report_tree,y_pred_tree,target_test_tree = scoring(y_tr,y_test,predictions_tr,predictions_test,0,'DecisionTree')


Accuracy: 0.5967741935483871
Confusion Matrix:
[[11 10]
 [15 26]]
Classification Report:
              precision    recall  f1-score   support

           0       0.42      0.52      0.47        21
           1       0.72      0.63      0.68        41

    accuracy                           0.60        62
   macro avg       0.57      0.58      0.57        62
weighted avg       0.62      0.60      0.61        62



In [12]:
accuracy_rf,conf_matrix_rf,class_report_rf,y_pred_rf,target_test_rf = scoring(y_tr,y_test,predictions_tr,predictions_test,0,"RandomF")

Accuracy: 0.7903225806451613
Confusion Matrix:
[[10 11]
 [ 2 39]]
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.48      0.61        21
           1       0.78      0.95      0.86        41

    accuracy                           0.79        62
   macro avg       0.81      0.71      0.73        62
weighted avg       0.80      0.79      0.77        62



In [13]:
accuracy_svm,conf_matrix_svm,class_report_svm,y_pred_svm,target_test_svm = scoring(y_tr,y_test,predictions_tr,predictions_test,0,"SVM")

Accuracy: 0.8225806451612904
Confusion Matrix:
[[12  9]
 [ 2 39]]
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.57      0.69        21
           1       0.81      0.95      0.88        41

    accuracy                           0.82        62
   macro avg       0.83      0.76      0.78        62
weighted avg       0.83      0.82      0.81        62

